# Import

In [47]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.transform import rotate, AffineTransform, warp
from skimage.util import random_noise
from skimage.filters import gaussian

# for creating validation set
from sklearn import model_selection

# for evaluating the model
from sklearn import metrics
from tqdm import tqdm
import gc

# PyTorch libraries and modules
import torch
from torch.utils import data
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Load Dataframe

In [48]:
train_df = pd.read_csv('./esiea_train_2022_DL.txt', header=None, names=['id'])
test_df = pd.read_csv('./esiea_valid_2022_DL.txt', header=None, names=['id', 'label'])
train_df[['id','label']] = train_df.id.str.split(expand=True)

In [49]:
train_df

,id,label
0,000002.jpg,1
1,000004.jpg,1
2,000005.jpg,1
3,000006.jpg,1
4,000007.jpg,1
...,...,...
102665,122666.jpg,1
102666,122667.jpg,1
102667,122668.jpg,1
102668,122669.jpg,-1


In [50]:
test_df

,id,label
0,078875.jpg,NaN
1,054206.jpg,NaN
2,039464.jpg,NaN
3,088046.jpg,NaN
4,114912.jpg,NaN
...,...,...
19995,097212.jpg,NaN
19996,050387.jpg,NaN
19997,049930.jpg,NaN
19998,054650.jpg,NaN


In [52]:
test_df['label'] = test_df['label'].fillna(2)
test_df

,id,label
0,078875.jpg,2.0
1,054206.jpg,2.0
2,039464.jpg,2.0
3,088046.jpg,2.0
4,114912.jpg,2.0
...,...,...
19995,097212.jpg,2.0
19996,050387.jpg,2.0
19997,049930.jpg,2.0
19998,054650.jpg,2.0


In [53]:
train, test =  model_selection.train_test_split(train_df, test_size = 0.2, random_state = 13)

In [54]:
train

,id,label
46676,055618.jpg,1
49981,059566.jpg,1
71784,085671.jpg,1
73752,088028.jpg,1
49395,058858.jpg,1
...,...,...
87796,104882.jpg,1
33634,040082.jpg,1
98378,117546.jpg,1
47280,056337.jpg,1


In [55]:
test

,id,label
58796,070095.jpg,1
14635,017464.jpg,-1
974,001156.jpg,1
15996,019106.jpg,1
88531,105767.jpg,1
...,...,...
45293,053989.jpg,1
37979,045281.jpg,1
55573,066247.jpg,-1
43549,051920.jpg,1


# Dataset

In [57]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, label_list, directory):
        self.label_list = label_list
        self.directory = directory

    def __len__(self):
        return len(self.label_list)

    def __getitem__(self, index):
        D = self.label_list.iloc[index]['id']
        X = imread(self.directory + self.label_list.iloc[index]['id'])
        X = torch.tensor(X, dtype=torch.float32)
        X/= 255
        X = torch.moveaxis(X, 2, 0)
        X = torchvision.transforms.functional.rgb_to_grayscale(X)
        y = int(self.label_list.iloc[index]['label'])
        y = 0 if y == -1 else y
        y = torch.tensor(y, dtype=torch.int64)
        return D, X, y
    
    def test(self, index):
        D = self.label_list.iloc[index]['id']
        X = imread(self.directory + self.label_list.iloc[index]['id'])
        X = X/255
        y = self.label_list.iloc[index]['label']

        return D, X, y
    
    def valid(self, index):
        D = self.label_list.iloc[index]['id']
        X = imread(self.directory + self.label_list.iloc[index]['id'])
        X = X/255
        return D, X

In [59]:
batch_size = 32
n_iters = 3000
print(len(train))
print(num_epochs)

params = {'batch_size': 100,
          'shuffle': True,
          'num_workers': 0}

# Generators
training_set = Dataset(train, "./img_esiea/")
train_loader = torch.utils.data.DataLoader(training_set,**params)

test_set = Dataset(test, "./img_esiea/")
test_loader = torch.utils.data.DataLoader(test_set, **params)

valid_set = Dataset(test_df, "./img_esiea/")
valid_loader = torch.utils.data.DataLoader(valid_set, **params)

82136
20


# Model

In [60]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

        # Max pool 2
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(16384, 2)

    def forward(self, x):
        #__import__("pdb").set_trace()
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)
        
        out = self.cnn3(out)
        out = self.relu3(out)

        # Max pool 2 
        out = self.maxpool3(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)

        return out

In [61]:
model = CNNModel()

In [62]:
model


CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu3): ReLU()
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=2, bias=True)
)

# Train

In [63]:
criterion = nn.CrossEntropyLoss()

In [64]:
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [66]:
def train():
    learning_rate = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(11):

        for i, (num, images, labels) in enumerate(train_loader):
            # Load images

            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()

            outputs = model(images.float())

            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)

            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            optimizer.step()

        print('epoch = ', epoch)

train()

epoch =  0
epoch =  1
epoch =  2
epoch =  3
epoch =  4
epoch =  5
epoch =  6
epoch =  7
epoch =  8
epoch =  9
epoch =  10


# Test

In [134]:
def test(): 
    correct = 0
    total = 0
    pred = []
    lab = []
    irr = []
    model.eval()
    # Iterate through test dataset
    for i, (num,images, labels) in enumerate(test_loader):
        # Load images
        #images = images.requires_grad_()

        outputs = model(images.float())
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        pred.append(predicted)
        lab.append(labels)
        correct += (predicted == labels).sum()
        irr.append(irr)

    accuracy = 100 * correct / total
    # Print Loss
    return  lab, pred, correct, total, accuracy
    #print('pred_liste : ', pred)
    #print('correct : ', str(correct))
    #print('total : ', str(total))
    #print('Accuracy: {}'.format(accuracy))

result_test = test()
print (result_test)
print('pred_liste : ', result_test[1])
print('correct : ', str(result_test[2]))
print('total : ', str(result_test[3]))
print('Accuracy: {}'.format(result_test[4]))


([tensor([0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 1, 0]), tensor([0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
        0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 0, 1]), tensor([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 

In [138]:
# get predicted results in a list
list_true = result_test[0]
list_pred = result_test[1]
liste_label_true = []
for i in range(len(list_true)):
    for j in range(len(list_true[i])):
        liste_label_true.append(list_true[i][j])
print(list(np.array(liste_label_true)))

[0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 

In [139]:
# get predicted results in a list
liste_label_pred = []
for i in range(len(list_pred )):
    for j in range(len(list_pred[i])):
        liste_label_pred.append(list_pred[i][j])
print(list(np.array(liste_label_pred)))

[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 

# Metrique

In [151]:
BC_0 = metrics.precision_score(liste_label_true, liste_label_pred, labels = [0], average='macro')
MC_0 = 1 - BC_0
print(BC_0)

0.7551447506103941


In [152]:
BC_1 = metrics.precision_score(liste_label_true, liste_label_pred, labels = [1], average='macro')
MC_1 = 1 - BC_1
print(BC_1)

0.8424324324324325


In [154]:
MC = [ MC_0, MC_1]
print('la moyenne des erreurs sur chaque classe :  {}'.format(MC))

la moyenne des erreurs sur chaque classe :  [0.2448552493896059, 0.1575675675675675]


In [146]:
target_names = ['class 0', 'class 1']
print(metrics.classification_report(liste_label_true, liste_label_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.76      0.65      0.70      6662
     class 1       0.84      0.90      0.87     13872

    accuracy                           0.82     20534
   macro avg       0.80      0.77      0.78     20534
weighted avg       0.81      0.82      0.81     20534



# Valid

In [85]:
def valid(): 
    total = 0
    liste = []
    pred = []
    lab = []
    model.eval()
    # Iterate through test dataset
    for i, (num,images, labels) in enumerate(valid_loader):
        # Load images
        #images = images.requires_grad_()

        outputs = model(images.float())
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Total number of labels
        #total += labels.size(0)

        # Total correct predictions
        liste.append(num)
        pred.append(predicted)
        
    # Print Loss
    return liste , pred
result = valid()
list_id = result[0]
list_label = result[1]

In [86]:
print(len(list_id))
print(len(list_id[1]))
print(len(list_label))
print(len(list_label[1]))

200
100
200
100


In [97]:
# get id in a list
liste_id_image = []
for i in range(len(list_id)):
    for j in range(len(list_id[i])):
        liste_id_image.append(list_id[i][j])
print(list(np.array(liste_id_image)))

['036075.jpg', '028892.jpg', '091061.jpg', '109600.jpg', '119124.jpg', '053330.jpg', '013252.jpg', '061318.jpg', '088305.jpg', '041741.jpg', '007036.jpg', '105144.jpg', '094067.jpg', '077225.jpg', '108853.jpg', '000549.jpg', '014183.jpg', '113672.jpg', '080955.jpg', '003366.jpg', '016548.jpg', '023832.jpg', '121199.jpg', '057956.jpg', '098949.jpg', '043223.jpg', '000952.jpg', '060667.jpg', '112300.jpg', '005213.jpg', '098554.jpg', '106113.jpg', '067601.jpg', '082983.jpg', '053562.jpg', '059771.jpg', '073074.jpg', '084378.jpg', '085691.jpg', '056706.jpg', '016730.jpg', '120286.jpg', '058059.jpg', '037533.jpg', '086955.jpg', '102012.jpg', '041457.jpg', '018766.jpg', '121483.jpg', '087062.jpg', '019241.jpg', '042188.jpg', '049682.jpg', '057592.jpg', '003459.jpg', '021231.jpg', '072441.jpg', '092090.jpg', '003547.jpg', '111454.jpg', '101818.jpg', '057281.jpg', '042194.jpg', '044742.jpg', '089155.jpg', '002842.jpg', '100726.jpg', '068652.jpg', '110359.jpg', '089558.jpg', '118200.jpg', '0962

In [105]:
# predicted result in a list
liste_label_image = []
for i in range(len(list_label)):
    for j in range(len(list_label[i])):
        liste_label_image.append(list_label[i][j])
x = liste_label_image
x = np.array(x)
x = list(x)
print(type(x))
print(list(x))

<class 'list'>
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 

In [125]:
# assemble id and predicted result
valid_fdf = pd.DataFrame(list(zip(liste_id_image, x)), columns =['id','label'])
valid_fdf

,id,label
0,036075.jpg,1
1,028892.jpg,0
2,091061.jpg,0
3,109600.jpg,0
4,119124.jpg,0
...,...,...
19995,081197.jpg,1
19996,067898.jpg,1
19997,078859.jpg,0
19998,017259.jpg,1


In [126]:
# change label 0 to -1
valid_fdf.loc[valid_fdf['label'] == 0, 'label'] = -1

In [115]:
# get valid file into dataframe (to have to same order as the orignal one)
valid_df = pd.read_csv('./esiea_valid_2022_DL.txt', header=None, names=['id', 'label'])
valid_df

,id,label
0,078875.jpg,NaN
1,054206.jpg,NaN
2,039464.jpg,NaN
3,088046.jpg,NaN
4,114912.jpg,NaN
...,...,...
19995,097212.jpg,NaN
19996,050387.jpg,NaN
19997,049930.jpg,NaN
19998,054650.jpg,NaN


In [131]:
# create the finale dataframe to have result in the same order as the given one
liste = []
l = list(valid_df['id'])

for index, row in valid_df.iterrows():
    liste.append(valid_fdf.loc[valid_fdf['id'] == row[0]]['label'].values[0]) # get label value
    
final_df =  pd.DataFrame(list(zip(l, liste)), columns =['id','label']) # assemble id and value in a df
final_df

,id,label
0,078875.jpg,-1
1,054206.jpg,1
2,039464.jpg,1
3,088046.jpg,-1
4,114912.jpg,1
...,...,...
19995,097212.jpg,1
19996,050387.jpg,1
19997,049930.jpg,1
19998,054650.jpg,-1


In [119]:
torch.save(model.state_dict(), 'model_da_epoch_10.pth') # savec model

In [133]:
# create csv file with the predicted value
final_df.to_csv(r'Nom_d’image predictions_HSIA_TILLACOVINDARADJANE.txt', header=None, index=None, sep=' ', mode='a')